# Classifieur binaire de champignons avec Xception (comestible / non comestible) : tuning du learning_rate et de l'optimizer

## dans ce notebook :
- entrainement d'un modèle avec un modèle Xception de base
- optimisation du modèle Xception (choix du learning_rate et de l'optimizer

Les inputs de ce notebook sont :
- un dossier d'images contenant l'ensemble des images disponibles
- un dataframe ou un fichier .csv contenant 2 colonnes :
    - la première 'filepath' avec les noms des fichiers images,
    - la seconde 'label' avec le nom des labels à identifier (0 = inedible, 1 = edible)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import cv2
import tensorflow as tf

from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, InputLayer, GlobalAveragePooling2D, Dropout
from tensorflow.keras import optimizers



## Préparation des données

In [ ]:
# import des données d'entrée
train_df = pd.read_csv(r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\reduced_dataset\df_train.csv')
val_df = pd.read_csv(r'C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\reduced_dataset\df_test.csv')
train_df.head()

In [ ]:
# Visualisation des distributions des labels dans les différents jeux de données
dataframes = ('train_df', 'val_df')
counts = {
    'inedible' : np.array([train_df.label.value_counts()[1], val_df.label.value_counts()[1]]),
    'edible' : np.array([train_df.label.value_counts()[0], val_df.label.value_counts()[0]])
}
width = 0.5
fig, ax = plt.subplots()
bottom = np.zeros(2)
for boolean, counts in counts.items():
    p = ax.bar(dataframes, counts, width, label=boolean, bottom=bottom)
    bottom += counts
    
ax.set_title('Labels distribution for differents balanced datasets')
ax.legend(loc='upper right')
plt.show()

In [ ]:
# définition de quelques paramètres
IMAGE_DIR = r"C:\Users\renamedadmin\Documents\Formation_Datascience\Projet_Datascientest_Champignons\Dossier_technique\02_Pieces_constitutives\reduced_dataset\full"
W, H = 299, 299
batch_size = 32
SEED = 3
epochs = 20

In [ ]:
# Création d'un data generator pour le dataset d'entrainement
train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        rotation_range = 30,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 0.2,
        horizontal_flip = True
        )

train_df["label"] = train_df["label"].apply(str)

train_generator = train_datagen.flow_from_dataframe(train_df,
                                                    IMAGE_DIR,
                                                    x_col="filename",
                                                    y_col="label",
                                                    target_size=(W,H),
                                                    class_mode="categorical",
                                                    batch_size=batch_size,
                                                    shuffle=True,
                                                    seed=SEED)

# Création d'un data generator pour le dataset de validation
val_datagen = ImageDataGenerator(
       preprocessing_function = preprocess_input)

val_df["label"] = val_df["label"].apply(str)
val_generator = val_datagen.flow_from_dataframe(val_df,
                                                IMAGE_DIR,
                                                x_col="filename",
                                                y_col="label",
                                                target_size=(W,H),
                                                class_mode="categorical",
                                                batch_size=batch_size)

In [ ]:
# Observation du fonctionnement du data generator sur quelques images du dataset d'entrainement
ex_df = train_df.sample(n=15).reset_index(drop=True)
ex_gen = train_datagen.flow_from_dataframe(ex_df,IMAGE_DIR,x_col="filename", y_col="label",
                                           target_size=(W,H), class_mode="categorical")


plt.figure(figsize=(15,15))
for i in range(0, 9):
    plt.subplot(5,3,i+1)
    for x, y in ex_gen:
        im = x[0]
        plt.imshow(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        break
plt.tight_layout()
plt.show()

In [ ]:
def plot_scores1(model)  :
    
    
    plt.figure(figsize=(12,4))

    plt.subplot(121)
    plt.plot(model.history['loss'])
    plt.plot(model.history['val_loss'])
    plt.title('loss by epoch')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='right')

    plt.subplot(122)
    plt.plot(model.history['accuracy'])
    plt.plot(model.history['val_accuracy'])
    plt.title('accuracy by epoch')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='right')
    plt.show()

## Xception de base

In [ ]:
xception = Xception(weights='imagenet', include_top=False)
xception.trainable = False
for layer in xception.layers: 
    layer.trainable = False
model = Sequential()
model.add(xception) # Ajout du modèle VGG16
model.add(GlobalAveragePooling2D()) 
model.add(Dense(1024,activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(rate=0.2))
model.add(Dense(2, activation='softmax'))
model.summary()

In [ ]:
# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)

# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])


In [ ]:
# recherche de possibilité d'utilisation d'une GPU
print('GPU Available : ', tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
# entrainement du modèle xception de base
history_xception = model.fit_generator(train_generator,
                                                  epochs=epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

In [ ]:
# affichage des metriques d'apprentissage
plot_scores1(history_xception)

## Optimisation des paramètres

### Learning rate

In [ ]:
# choix de quelques learning rate à évaluer
lr0 = 0.1
lr1 = 0.01
lr2 = 0.001
lr3 = 0.0001
lr4 = 0.00001
lr5 = 0.000001

# nombre d'epochs pour l'évaluation
eval_epochs = 10

# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = lr0, decay = 1e-6, momentum = 0.9, nesterov = True)
# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_lr0 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = lr1, decay = 1e-6, momentum = 0.9, nesterov = True)
# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_lr1 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = lr2, decay = 1e-6, momentum = 0.9, nesterov = True)
# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_lr2 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = lr3, decay = 1e-6, momentum = 0.9, nesterov = True)
# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_lr3 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = lr4, decay = 1e-6, momentum = 0.9, nesterov = True)
# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_lr4 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

# définition de l'optimizer
sgd = optimizers.SGD(learning_rate = lr5, decay = 1e-6, momentum = 0.9, nesterov = True)
# compilation du modèle
model.compile(optimizer=sgd, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_lr5 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size
                                                  )

In [ ]:
# affichage graphique des résultats

width = 0.5
plt.figure(figsize=(10,8))
plt.subplot(221)
plt.plot(history_lr0.history['loss'], label = 'lr = 0.1')
plt.plot(history_lr1.history['loss'], label = 'lr = 0.01')
plt.plot(history_lr2.history['loss'], label = 'lr = 0.001')
plt.plot(history_lr3.history['loss'], label = 'lr = 0.0001')
plt.plot(history_lr4.history['loss'], label = 'lr = 0.00001')
plt.plot(history_lr5.history['loss'], label = 'lr = 0.000001')
plt.title('loss by epoch for training')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['lr = 0.1', 'lr = 0.01', 'lr = 0.001', 'lr = 0.0001', 'lr = 0.00001', 'lr = 0.000001'], loc='upper right')

plt.subplot(222)
plt.plot(history_lr0.history['val_loss'], label = 'lr = 0.1')
plt.plot(history_lr1.history['val_loss'], label = 'lr = 0.01')
plt.plot(history_lr2.history['val_loss'], label = 'lr = 0.001')
plt.plot(history_lr3.history['val_loss'], label = 'lr = 0.0001')
plt.plot(history_lr4.history['val_loss'], label = 'lr = 0.00001')
plt.plot(history_lr5.history['val_loss'], label = 'lr = 0.000001')
plt.title('loss by epoch for validation')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['lr = 0.1', 'lr = 0.01', 'lr = 0.001', 'lr = 0.0001', 'lr = 0.00001', 'lr = 0.000001'], loc='lower right')

plt.subplot(223)
plt.plot(history_lr0.history['accuracy'], label = 'lr = 0.1')
plt.plot(history_lr1.history['accuracy'], label = 'lr = 0.01')
plt.plot(history_lr2.history['accuracy'], label = 'lr = 0.001')
plt.plot(history_lr3.history['accuracy'], label = 'lr = 0.0001')
plt.plot(history_lr4.history['accuracy'], label = 'lr = 0.00001')
plt.plot(history_lr5.history['accuracy'], label = 'lr = 0.000001')
plt.title('accuracy by epoch for training')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['lr = 0.1', 'lr = 0.01', 'lr = 0.001', 'lr = 0.0001', 'lr = 0.00001', 'lr = 0.000001'], loc='lower right')

plt.subplot(224)
plt.plot(history_lr0.history['val_accuracy'], label = 'lr = 0.1')
plt.plot(history_lr1.history['val_accuracy'], label = 'lr = 0.01')
plt.plot(history_lr2.history['val_accuracy'], label = 'lr = 0.001')   
plt.plot(history_lr3.history['val_accuracy'], label = 'lr = 0.0001')     
plt.plot(history_lr4.history['val_accuracy'], label = 'lr = 0.00001') 
plt.plot(history_lr5.history['val_accuracy'], label = 'lr = 0.000001') 
plt.title('accuracy by epoch for validation')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['lr = 0.1', 'lr = 0.01', 'lr = 0.001', 'lr = 0.0001', 'lr = 0.00001', 'lr = 0.000001'], loc='lower right')
plt.show()

### Optimizer 

In [ ]:
# choix de quelques learning rate à évaluer
opt0 = optimizers.SGD()
opt1 = optimizers.Adadelta()
opt2 = optimizers.Adagrad()
opt3 = optimizers.Adamax()
opt4 = optimizers.Ftrl()
opt5 = optimizers.Nadam()
opt6 = optimizers.RMSprop()

# nombre d'epochs pour l'évaluation
eval_epochs = 10

# compilation du modèle
model.compile(optimizer=opt0, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt0 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

# compilation du modèle
model.compile(optimizer=opt1, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt1 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

# compilation du modèle
model.compile(optimizer=opt2, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt2 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

# compilation du modèle
model.compile(optimizer=opt3, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt3 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

# compilation du modèle
model.compile(optimizer=opt4, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt4 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

# compilation du modèle
model.compile(optimizer=opt5, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt5 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

# compilation du modèle
model.compile(optimizer=opt6, loss = "categorical_crossentropy", metrics=["accuracy"])
# entrainement du modèle xception de base
history_opt6 = model.fit_generator(train_generator,
                                                  epochs=eval_epochs,
                                                  validation_data=val_generator,
                                                  validation_steps=len(val_df)//batch_size,
                                                  steps_per_epoch=len(train_df)//batch_size)

In [ ]:
# affichage graphique des résultats sur le choix de l'optimizer

plt.figure(figsize=(10,8))
plt.subplot(221)
plt.plot(history_opt0.history['loss'], label = 'SGD')
plt.plot(history_opt1.history['loss'], label = 'Adadelta')
plt.plot(history_opt2.history['loss'], label = 'Adagrad')
plt.plot(history_opt3.history['loss'], label = 'Adamax')
plt.plot(history_opt4.history['loss'], label = 'Ftrl')
plt.plot(history_opt5.history['loss'], label = 'Nadam')
plt.plot(history_opt6.history['loss'], label = 'RMSprop')
plt.title('loss by epoch for training')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['SGD', 'Adadelta', 'Adagrad', 'Adamax', 'Ftrl', 'Nadam', 'RMSprop'], loc='lower right')

plt.subplot(222)
plt.plot(history_opt0.history['val_loss'], label = 'SGD')
plt.plot(history_opt1.history['val_loss'], label = 'Adadelta')
plt.plot(history_opt2.history['val_loss'], label = 'Adagrad')
plt.plot(history_opt3.history['val_loss'], label = 'Adamax')
plt.plot(history_opt4.history['val_loss'], label = 'Ftrl')
plt.plot(history_opt5.history['val_loss'], label = 'Nadam')
plt.plot(history_opt6.history['val_loss'], label = 'RMSprop')
plt.title('loss by epoch for validation')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['SGD', 'Adadelta', 'Adagrad', 'Adamax', 'Ftrl', 'Nadam', 'RMSprop'], loc='lower right')

plt.subplot(223)
plt.plot(history_opt0.history['accuracy'], label = 'SGD')
plt.plot(history_opt1.history['accuracy'], label = 'Adadelta')
plt.plot(history_opt2.history['accuracy'], label = 'Adagrad')
plt.plot(history_opt3.history['accuracy'], label = 'Adamax')
plt.plot(history_opt4.history['accuracy'], label = 'Ftrl')
plt.plot(history_opt5.history['accuracy'], label = 'Nadam')
plt.plot(history_opt6.history['accuracy'], label = 'RMSprop')
plt.title('accuracy by epoch for training')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['SGD', 'Adadelta', 'Adagrad', 'Adamax', 'Ftrl', 'Nadam', 'RMSprop'], loc='lower right')

plt.subplot(224)
plt.plot(history_opt0.history['val_accuracy'], label = 'SGD')
plt.plot(history_opt1.history['val_accuracy'], label = 'Adadelta')
plt.plot(history_opt2.history['val_accuracy'], label = 'Adagrad')
plt.plot(history_opt3.history['val_accuracy'], label = 'Adamax')
plt.plot(history_opt4.history['val_accuracy'], label = 'Ftrl')
plt.plot(history_opt5.history['val_accuracy'], label = 'Nadam')
plt.plot(history_opt6.history['val_accuracy'], label = 'RMSprop')
plt.title('accuracy by epoch for validation')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['SGD', 'Adadelta', 'Adagrad', 'Adamax', 'Ftrl', 'Nadam', 'RMSprop'], loc='lower right')
plt.show()